In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [2]:
# Helper functions
def stripText(text):
    x =[]
    return x.append(text)
# Convert function to UDF
stripTextUDF = f.udf(lambda z: stripText(z))

In [3]:
spark = SparkSession.builder \
    .appName("Redit Summarization")\
    .master("yarn")\
    .config("spark.executor.memoryOverhead","2048")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.2.3,org.apache.hadoop:hadoop-aws:2.7.3")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19edf4d9-8de7-41e8-8709-5dd689b56ce2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.2.3 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastu

:: retrieving :: org.apache.spark#spark-submit-parent-19edf4d9-8de7-41e8-8709-5dd689b56ce2
	confs: [default]
	0 artifacts copied, 84 already retrieved (0kB/25ms)
2022-04-23 17:50:19,358 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-23 17:50:21,357 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-04-23 17:50:23,217 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-04-23 17:50:40,849 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-3.2.3.jar added multiple times to distributed cache.
2022-04-23 17:50:40,849 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.apache.hadoop_ha

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [4]:
# Read data
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/dis_materials/data_reddit.csv")

In [5]:
# drop duplicates
df2 = df.dropDuplicates()

In [6]:
# drop null values
df2 = df2.na.drop(subset=["subreddit","subreddit_id","body","ups"])

In [7]:
df_english = df2.drop("name","score_hidden","id","removal_reason","gilded","retrieved_on","edited","controversiality")

In [8]:
# df_english.withColumn('created_utc', f.date_format(df_english.created_utc.cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

In [9]:
df_english= df_english.withColumn('created_utc', f.from_unixtime('created_utc').cast(t.DateType()))

In [10]:
df_english = df_english.withColumn("ups", df_english["ups"].cast(t.IntegerType()))

In [11]:
channel = "namenerds"

In [12]:
df_channel = df_english.filter(df_english.subreddit == channel)

In [13]:
# df_channel = df_channel.withColumn("body_list",stripTextUDF(f.col("body")))

NameError: name 'col' is not defined

In [ ]:
df_channel.collect()

In [ ]:
# df_channel.select(f.split(f.col("body"),"").alias("body_list")).take(5)

In [14]:
df_top_comments_ids = df_channel.filter(df_channel.link_id == df_channel.parent_id).rdd.map(lambda x: x.link_id).collect()

In [15]:
df_top_comments_ids

['t3_34gpda',
 't3_34gp4s',
 't3_34gp4s',
 't3_34gp4s',
 't3_345hx3',
 't3_34g8l0',
 't3_34gp4s',
 't3_34gpda',
 't3_34g8l0',
 't3_34h0zj',
 't3_34gp4s',
 't3_34h0zj',
 't3_34gp4s',
 't3_34h0zj',
 't3_34efs2',
 't3_34fp02',
 't3_34h0zj',
 't3_34g8l0',
 't3_34g8l0',
 't3_34hsve',
 't3_34be76',
 't3_34gp4s',
 't3_34gpda',
 't3_34gpda',
 't3_34hsve',
 't3_34jidd',
 't3_34j7q0',
 't3_34ixde',
 't3_34ixde',
 't3_34g8l0',
 't3_33rk3i',
 't3_34j7q0',
 't3_34jidd',
 't3_34gp4s',
 't3_34ixde',
 't3_34g8l0',
 't3_34j7q0',
 't3_34kn0s',
 't3_34lpy0',
 't3_34lw4m',
 't3_34lw4m',
 't3_34lpy0',
 't3_34lpy0',
 't3_34lw4m',
 't3_34kgof',
 't3_34lpy0',
 't3_34g8l0',
 't3_34lw4m',
 't3_34lpy0',
 't3_34kgof',
 't3_34lpy0',
 't3_34hsve',
 't3_34ohdt',
 't3_34o3m2',
 't3_34o3m2',
 't3_34ohdt',
 't3_34o3m2',
 't3_34clgs',
 't3_34lpy0',
 't3_34o3m2',
 't3_34oba5',
 't3_34o3m2',
 't3_34oba5',
 't3_34o3m2',
 't3_34o3m2',
 't3_34o3m2',
 't3_34o3m2',
 't3_34oba5',
 't3_34o3m2',
 't3_34oba5',
 't3_34oba5',
 't3_3

In [16]:
df_channel.dtypes

[('created_utc', 'date'),
 ('ups', 'int'),
 ('subreddit_id', 'string'),
 ('link_id', 'string'),
 ('author_flair_css_class', 'string'),
 ('author_flair_text', 'string'),
 ('subreddit', 'string'),
 ('downs', 'string'),
 ('archived', 'string'),
 ('author', 'string'),
 ('score', 'string'),
 ('body', 'string'),
 ('distinguished', 'string'),
 ('parent_id', 'string')]

In [17]:
df_top_comments_n_replies = df_channel.filter(df_channel.parent_id.isin(df_top_comments_ids))

In [18]:
df = df_top_comments_n_replies.groupby("created_utc","subreddit","parent_id").\
    agg(sum("ups").alias("total_upvotes"),\
    max("total_upvotes").alias("max_ups"))

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
df.show()

In [ ]:
# df_top_comments_n_replies =  df_top_comments_n_replies.groupby("created_utc","subreddit","parent_id").sum("sum(ups)").alias("total_upvotes").show()

In [ ]:
# df = df_top_comments_n_replies.groupBy("subreddit").agg(f.flattenList(f.collect_list("body_list")).alias("comment_replies"))

In [ ]:
# df.select(transform("comment_replies", flattenList).alias("comment_replies")).show()

2022-04-21 19:25:50,179 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 2 for reason Container marked as failed: container_1644567623435_0087_01_000004 on host: datanode2. Exit status: -100. Diagnostics: Container released on a *lost* node.
2022-04-21 19:25:50,187 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 1 for reason Container marked as failed: container_1644567623435_0087_01_000003 on host: datanode2. Exit status: -100. Diagnostics: Container released on a *lost* node.
2022-04-21 19:25:50,188 ERROR cluster.YarnScheduler: Lost executor 2 on datanode2: Container marked as failed: container_1644567623435_0087_01_000004 on host: datanode2. Exit status: -100. Diagnostics: Container released on a *lost* node.
2022-04-21 19:25:50,191 ERROR cluster.YarnScheduler: Lost executor 1 on datanode2: Container marked as failed: container_1644567623435_0087_01_000003 on host: datanode2. Exit status: -100